In [5]:
%pip install

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: You must give at least one requirement to install (see "pip help install")


In [8]:
import pandas as pd
import requests

# USGSから過去1か月の地震データを取得
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
response = requests.get(url)

# データをファイルに保存
with open('earthquakes.csv', 'wb') as file:
    file.write(response.content)

# データをDataFrameに読み込む
data = pd.read_csv('earthquakes.csv')

# データの最初の5行を表示
data.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-11-12T05:17:34.037Z,62.363300,-150.903700,21.60,1.10,ml,NaN,NaN,NaN,0.42,...,2025-11-12T05:19:40.556Z,"16 km SSW of Petersville, Alaska",earthquake,NaN,0.30,NaN,NaN,automatic,ak,ak
1,2025-11-12T05:05:16.120Z,38.821835,-122.805168,2.37,0.52,md,6.0,95.0,0.01585,0.01,...,2025-11-12T05:22:19.369Z,"7 km NW of The Geysers, CA",earthquake,1.15,5.90,0.290,8.0,automatic,nc,nc
2,2025-11-12T05:01:50.980Z,33.458833,-116.440667,2.40,0.76,ml,47.0,74.0,0.13970,0.19,...,2025-11-12T05:05:17.178Z,"23 km NNW of Borrego Springs, CA",earthquake,0.17,0.66,0.131,25.0,automatic,ci,ci
3,2025-11-12T04:58:07.336Z,61.556300,-147.875700,20.60,1.80,ml,NaN,NaN,NaN,0.76,...,2025-11-12T05:10:11.125Z,"30 km SSW of Glacier View, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
4,2025-11-12T04:57:17.298Z,31.463000,-104.346000,0.00,1.50,ml,12.0,249.0,0.20000,0.90,...,2025-11-12T05:00:19.335Z,"55 km WNW of Toyah, Texas",earthquake,0.00,NaN,0.200,7.0,automatic,tx,tx


In [11]:
import pandas as pd
import folium
from datetime import datetime, timedelta
from io import StringIO

csv_data = """time,latitude,longitude,depth,mag,place
2025-11-05T12:34:00Z,38.1,142.3,35,5.8,"off the coast of Miyagi, Japan"
2025-11-02T08:12:00Z,35.7,139.7,10,3.4,"near Tokyo, Japan"
2025-11-03T10:00:00Z,37.8,144.5,20,6.2,"off the coast of Fukushima, Japan"
2025-11-01T20:45:00Z,34.1,135.5,50,2.8,"Osaka, Japan"
2025-11-06T14:10:00Z,-10.2,162.4,70,7.1,"Solomon Islands"
2025-11-07T03:30:00Z,-20.0,175.0,250,5.9,"Fiji region"
2025-11-08T05:00:00Z,-15.0,167.0,400,6.5,"Vanuatu"
"""

data = pd.read_csv(StringIO(csv_data))

data["japan_time"] = pd.to_datetime(data["time"]) + timedelta(hours=9)
data["japan_time"] = data["japan_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

def get_color(mag):
    if mag < 3.0:
        return "green"
    elif 3.0 <= mag <= 6.0:
        return "orange"
    else:
        return "red"

m = folium.Map(location=[36.0, 138.0], zoom_start=4)

for _, row in data.iterrows():
    color = get_color(row["mag"])
    tooltip = (
        f"<b>{row['place']}</b><br>"
        f"マグニチュード: {row['mag']}<br>"
        f"深さ: {row['depth']} km<br>"
        f"日時(日本時間): {row['japan_time']}"
    )
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5 + row["mag"],  
        color=color,
        fill=True,
        fill_opacity=0.7,
        tooltip=tooltip
    ).add_to(m)

m.save("earthquakes_map.html")
m

日本語サマリー

小さな地震に気づいてないだけで、ほかの国に比べてたくさんの地震が起こっていることに驚いた。
地震に慣れてしまうと万が一の時の行動が遅れてしまうと思った。なので避難訓練の大切さに気付いた。
日本人の気づきにくい3.0以下がほかの国に比べて多かった。そして大きな被害をもたらす震度6.0以上も多く、簡単に避難できるように日ごろから建物の避難経路を確認したり、家の非常食の場所などを確認した。
四つのクレートの重なっている本島の真ん中あたりの地震が多い。